In [1]:
import pandas as pd
import scipy.sparse as sparse
import csv
from tqdm import tqdm
import implicit
from collections import defaultdict

In [5]:
# Load the data
raw_data = pd.read_csv("/Users/s0c02nj/Desktop/train- Capillary 4/train.csv")
test_data = pd.read_csv("/Users/s0c02nj/Desktop/train- Capillary 4/test_nFNPSyV.csv")

# Drop NaN columns
data = raw_data.dropna()
data = data.copy()

In [6]:
data.head()

,UserId,productid,Quantity,OrderDate
0,18075,12322648,1,01/04/18
1,6820,12371370,1,01/04/18
2,6820,12973004,1,01/04/18
3,6820,12657560,1,01/04/18
4,6820,11659914,1,01/04/18


In [7]:
sparse_item_user = sparse.csr_matrix((data['Quantity'].astype(float), (data['productid'], data['UserId'])))
sparse_user_item = sparse.csr_matrix((data['Quantity'].astype(float), (data['UserId'], data['productid'])))

In [8]:
data['OrderDate'] = pd.to_datetime(data.OrderDate)
data = data.sort_values('OrderDate')

In [10]:
#data.head()

In [11]:
# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=50, 
                                             regularization=0.2, 
                                             iterations=20, 
                                             calculate_training_loss=True)

In [12]:
# Calculate the confidence by multiplying it by alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

In [13]:
# Fit the model
model.fit(data_conf)

100%|██████████| 20.0/20 [00:48<00:00,  1.95s/it, loss=1.53e-06]


In [14]:
user_2_prod = defaultdict(list)

for user, prod in zip(data.UserId, data.productid):
    user_2_prod[user].append(prod)

In [16]:
# Create recommendations for all users
fields=['UserId', 'product_list']

userids = test_data['UserId'].drop_duplicates().values.tolist()
ans = []
count_last_purchase = 0
for user in tqdm(userids):
    
    #recommended = model.recommend(user, sparse_user_item)
    #prod_ids_rec = [obj[0] for obj in recommended]
    
    # Use implicit to get similar items.
    n_similar = 10
    similar = model.similar_items(user_2_prod[user][-1], n_similar)
    
    '''
    print("user", user_2_prod[user][-1])
    print(similar)
    
    print("Rec")
    print(recommended)
    '''
    prod_ids_sim = [obj[0] for obj in similar]
    prod_scores = [obj[1] for obj in similar]
        
    ans.append(prod_ids_sim)

  0%|          | 0/2350 [00:00<?, ?it/s]/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/numpy/linalg/linalg.py:2287: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
100%|██████████| 2350/2350 [33:21<00:00,  1.17it/s]


In [37]:
filename = 'item_similarity_als_factor_50_reg_1_iter_20_mul_15.csv'

In [38]:
sub = pd.DataFrame(columns=fields)

In [39]:
sub['UserId'] = userids
sub['product_list'] = ans

In [35]:
sub.tail(10)

,UserId,product_list
2340,27692,"[12423486, 11407290, 11146754, 11481106, 12445..."
2341,27701,"[12407052, 12423486, 11481106, 11407290, 11146..."
2342,27705,"[12407052, 12445178, 11407290, 11460608, 12437..."
2343,27715,"[12423486, 11407290, 11146754, 11481106, 12445..."
2344,27725,"[12423486, 11481106, 11407290, 11146754, 12445..."
2345,27727,"[12423486, 11660112, 11481106, 12437500, 11146..."
2346,27731,"[12423486, 11146754, 11407290, 11481106, 12445..."
2347,27755,"[12407052, 12423486, 11481106, 11407290, 11146..."
2348,27756,"[12423486, 11407290, 11481106, 11146754, 12445..."
2349,27766,"[12423486, 11481106, 11407290, 11146754, 12445..."


In [36]:
sub.to_csv(filename, index=False)